<a href="https://colab.research.google.com/github/JesusdelCas99/T_H_DNN_Masking/blob/main/Training_DNN_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
#Nos movemos al directorio deseado en google collab
%cd '/content/drive/MyDrive/Colab_Notebooks/sigmatools'
%ls 

/content/drive/MyDrive/Colab_Notebooks/sigmatools
F05_440C020E_CAF.CH5.noise.wav   F05_440C020E_CAF.CH5.wav  sigmatools/
F05_440C020E_CAF.CH5.speech.wav  setup.py                  sigmatools.egg-info/


In [3]:
# Preparación de los datos en el nodo para trabajar localmente (adaptar según sea necesario)
# !mkdir /tmp/TIMIT_300/
# !mkdir /tmp/TIMIT_300/Noises/
# !mkdir /tmp/TIMIT_300/Resultados/
# !cp -r /home/amgg/Desarrollo/TIMIT_PESQ/TIMIT_300/MONO /tmp/TIMIT_300/
# !cp /home/amgg/Desarrollo/TIMIT_PESQ_builder/Noises/*.wav /tmp/TIMIT_300/Noises/

In [4]:
####### DEFINICION DE NUESTRA CLASE DATASET (TRATAMIENTO DE LA BASE DE DATOS)

#Emplearemos como punto de partida la clase DataSet de Pytorch
from torch.utils.data import Dataset

# Necesitamos algunas librerias para ciertas tareas específicas
from scipy.io import wavfile      # Lectura de WAV files
import glob                       # Gestion de archivos
import os                         # Funciones del SO
import pickle                     # Lectura y salvado de objetos en disco


class TimitDataset(Dataset):
    # Particularizamos la Clase DataSet a nuestras necesidades: podemos añadir nuevos métodos o podemos REDEFINIR
    # los ya existentes (lo que se conoce como 'Herencia' en OOP)
    
    def __init__(self, dir_db, dir_noise, mode='MONO', conj='train'):
        # Definimos la inicialización del objeto con los parametros:
        #     dir_db   : (string) directorio donde está la base de datos a usar
        #     dir_noise: (string) directorio donde está el ruido
        #     mode     : (string) subdirectorio de la BB.DD. especifico para experimento
        #     conj     : (string) subset de la BB.DD. (train, valid o set) 
        
        #Definicion de los parámetros de la estructura
        self.dir_db = dir_db
        self.dir_noise = dir_noise
        self.mode = mode
        self.conj = conj

        # Estos datos son fijos para nuestros experimentos (aunque podrían ser tambien parametros, los hemos dejado
        # como constantes)
        self.snrs = [20, 15, 10, 5, 0, -5]                      # SNRs para el ruido
        self.noises = ['CARJA', 'BSTJA', 'PREST', 'STRAF']      # Tipos de ruido
        
        if self.conj == 'test':                                 # En TEST se añaden tipos de RUIDOS no vistos en Training
            #Basicamente hacemos un horzcat en caso de ser un cojunto de test, añadiendo nuevos parametros
            self.noises.extend(['PCAFE', 'PSTAT', 'PSTJA', 'TPBUS'])

        # Lee y organiza los ficheros de ruido en un diccionario
        self.sample_noises = self._get_dict_noises()
        
        # Obtiene un listado de todos los ficheros '.pkl' que hay en el subset seleccionado (barriendo subdirectorios) 
        self.lista_files = glob.glob(self.dir_db + self.mode + '/' + self.conj + '/*/*.pkl')


    def __len__(self):
        # Devuelve el total de número de ficheros en el subset de 'train','valid','set'
        return len(self.lista_files)
    
    
    def __getitem__(self, idx):
        #Construye un diccionario de una sola entrada con el sample a tratar de voz captada con ruido

        # Devuelve el 'sample' (ver más abajo) identificado por idx en la lista de ficheros del set
        file_meta = self.lista_files[idx]

        #Nombre del archivo en cuestion
        example_id = os.path.splitext(os.path.basename(file_meta))[0]

        # Carga el objeto de metadatos, que indica exactamente como se ha generado la señal de voz noisy del archivo
        with open(file_meta, 'rb') as f:
            metadata = pickle.load(f)
        n_ind = metadata['n_ind']
        G = metadata['G_0']
        v_norm = metadata['v0_norm']

        # Leemos la señal de voz noisy
        y = self._read_sample(file_meta[:-4] + '_CH0.wav')  # [:5*16000]

        label_noise = example_id.split('_')[2]
        n = self.sample_noises[label_noise][n_ind: n_ind + y.shape[0]] * G / v_norm

        # Con la información del objeto de metadatos restamos el ruido exacto, obteniendo la señal limpia 
        x = y - n

        # Construimos un "sample" de entrenamiento en forma de diccionario con los siguientes datos:
        sample = {'example_id': example_id, 'noisy': y, 'clean': x, 'noise': n, 'seq_len': len(y)}

        return sample
 

    def _read_sample(self, file_speech):
        # Lee el fichero WAV y devuelve un vector con sus muestras
        time_signal = wavfile.read(file_speech)[1] * 1.0
        return time_signal #Nos quedamos con el canal derecho


    def _get_dict_noises(self):
        # Organiza las muestras de ruido
        # Construye un diccionario con las muestras de los ruidos. El nombre de archivo identifica el tipo de ruido 
        # y el subset al que se debe aplicar (Training, Validacion, Test).
        
        sample_noises = dict() #Inicializamos el diccionario
        
        # Para saber que fichero hay que leer, debemos traducir el conjunto que queremos (e.g. 'train'->ficheros 'T')
        if self.conj == 'train':
            id_conj = 'T'
        elif self.conj == 'valid':
            id_conj = 'V'
        elif self.conj == 'test':
            id_conj = 'E'
        else:
            raise ValueError('Unexpected subset')

        for noise in self.noises:
            # Para cada uno de los noises (['CARJA', 'BSTJA', 'PREST',...]) lee el fichero WAV y guarda los datos 
            # en un diccionario en donde el id es el nombre del ruido
            noise_id = noise + '_' + self.mode + '_' + id_conj #Por ejemplo noise_id=CARJA_MONO_T
            n = self._read_sample(self.dir_noise + noise_id + '_CH1.wav') #Leemos el ruido de un determinado fichero
            sample_noises.update({noise: n})

        return sample_noises

In [5]:
####### CONSTRUCCION DE NUESTRA (CLASE DE) RED NEURONAL

import numpy as np

# Esta es una libreria propia con funciones diversas, de ella importamos las funciones para
# construir una ventana de hamming, calcular una transformada corta de Fourier y su inversa (síntesis)
from sigmatools.transform.stft_fn import stft, istft
from sigmatools.transform.window import hann_sqrt

import torch
import torch.nn as nn
import torch.nn.functional as F


# Primero construimos una clase generica de red neuronal orientada al Procesamiento de Voz y capaz de usar nuestro
# dataset (el que hemos definido antes)
class _base_net(nn.Module):
    # Aqui PARTICULARIZAMOS la clase nn.Module proporicionada por pytorch para la construcción de DNNs. 
    
    #Constructor
    def __init__(self, opt):
        # Definimos la inicialización de la DNN mediante un objeto de opciones (al poder ser muy distintas y
        # variadas las opciones que necesitemos, pasamos directamente un objeto que dará esta informacion)
        
        # IMPORTANTE, a diferencia de la clase DataSet, la clase Module tiene ya un INIT (para inicializar las 
        # 'tripas' de la red neuronal que ni nos importan ni nos interesan). Por eso llamamos aqui a la inicialización
        # de la clase padre (o superclase)

        #Importamos los parametros de la clase nn.Module
        super(_base_net, self).__init__()

        # Establecemos los atributos que toda DNN orientada al procesado de voz (tal y como lo vamos a hacer
        # nososotros, con STFT) debe de tener. Basicamente definimos los parámetros de la STFT
        
        self.window_length = opt.window_length                   # Tamaño de la ventana (tramas) empleada para la STFT
        self.shift = opt.shift                                   # Desplazamiento de ventanas (entre frames)
        self.window = hann_sqrt(self.window_length, self.shift)  # Ventana de analisis empleada para la STFT
        self.hidden_units = opt.hidden_units                     # Numero de unidades ocultas por capa (Numero de neuronas por capa)
        self.dropout_rate = opt.dropout_rate                     # Dropout rate a emplear en el entrenamiento

    
    def collate_fn(self, input_batch):
        # Definicion del batch

        # Toda DNN que construyamos debe ser capaz de proveer de una función para la carga de un batch de 'samples' 
        # de entrenamiento acorde a sus propias necesidades.
        
        # Recordemos cómo era un sample del dataset (clase anterior):
        #  # Construimos un "sample" de entrenamiento en forma de diccionario con los siguientes datos:
        #  sample = {'example_id': example_id, 'noisy': y, 'clean': x, 'noise': n, 'seq_len': len(y)}
        
        # Mas adelante veremos que pytorch prepara en paralelo (en la CPU, mientras la GPU está ocupada
        # entrenando la red neuronal con un batch) los datos del siguiente batch. Esta función le indica como
        # debe hacerse esta preparación (que no depende de los datos, sino de como hayamos diseñado nuestra DNN). 
        
        # El parametro input_bach no es más que una lista de 'sample' anteriores
        
        
        # Construimos tres listas para cada una de la informacion que nos interesa (ie. toda menos el 'example_id')
        list_seq_len = [item['seq_len'] for item in input_batch]     # Lista longitudes de las señales del batch
        list_clean = [item['clean'] for item in input_batch]         # Lista de vectores con señales limpias
        list_noisy = [item['noisy'] for item in input_batch]         # Lista de vectores con señales noisy

        
        # A partir de aqui lo que hacemos es construir el batch de entrenamiento a partir de las listas anteriore
        # (en el fondo a partir de la lista de 'samples' que hemos recibido a la entrada)
        seq_len = np.array(list_seq_len)

        # Padding para que todas las señales tengan la misma longitud
        max_length = np.amax(seq_len)
        clean_pad = np.stack([np.pad(item, (0, max_length - len(item)), 'constant') for item in list_clean])
        noisy_pad = np.stack([np.pad(item, (0, max_length - len(item)), 'constant') for item in list_noisy])

        # Calculo de la STFT (Magnitud)
        #ToDO: Calcular la STFT y su valor absoluto para voz limpia y ruidosa (variables clean y noisy)
        STFT_clean=abs(stft(clean_pad,self.window,size=self.window_length,shift=self.shift))

        STFT_noisy=abs(stft(noisy_pad,self.window,size=self.window_length,shift=self.shift))

        # Calculo de log-spectra
        #ToDo: Calcular el logritmo del espectro (cuidado con valores a cero) y aplicar normalizacion recursiva (mas abajo)
        delta=1e-12
        noisy_lps=np.log(STFT_noisy+delta)

        noisy_lps_norm=self.rec_mean_normalization(noisy_lps)
        
        frame_len = self.sample_to_frame(seq_len) #Numero de ventanas para cada señal dentro del batch

        pad_mask = np.zeros((noisy_lps.shape[0], noisy_lps.shape[1], 1))
        for idx in range(len(frame_len)):
            pad_mask[idx, :frame_len[idx]] = 1.0

        # Construcción del batch 'a huevo' para que lo use la DNN
        output_batch = {'features': torch.from_numpy(noisy_lps_norm).float(),
                        'pad_mask': torch.from_numpy(pad_mask).float(),
                        'noisy': torch.from_numpy(STFT_noisy).float(),
                        'clean': torch.from_numpy(STFT_clean).float()}

        return output_batch
    
    #ToDo: Crear una funcion collate similar, pero para usarla en test (i.e. simplificado y para un solo elemento)
    # ver ejercicio final

    # Resto de funciones auxiliares implicadas en la construccion de un batch:
    def sample_to_frame(self, nsample):
        #Note: This function assumes there is fading in the STFT computation --- NO ENTIENDO
        nframe = np.int_(np.ceil((((nsample - self.window_length) * 1.0) / self.shift)) + 3) * np.array([1])
        return nframe


    def istft(self, enh_stft, seq_len):
        #ToDo: Definir la ISTFT
        enh_signal = istft(enh_stft, self.window, signal_length=seq_len, size=self.window_length, shift=self.shift)
        return enh_signal


    def rec_mean_normalization(self, Y):
        # Create vector to save result and mean
        #ToDo: Un vector donde guardar ls resutlados y otro para ir almancenando la media (mirar zeros_like)
        # Señal Y tiene 3 dimensiones; batch, tiempo y frecuencia (nos interesa el tiempo)
        Y_norm = np.zeros_like(Y)
        means = np.copy(Y_norm[:,0,:])
        # Recursive mean computation and subtraction
        for t in range(Y.shape[1]):
            #ToDo: Calcular la media recursiva en cada frame (a partir de la media anterior y el valor actual)
            #ToDo: Sustraer la media en el instante actual
            means = t/(t+1.0) * means + 1.0/(t+1.0) * Y[:,t,:]
            Y_norm[:,t,:] = Y[:,t,:] - means
        return Y_norm
    ##########################
    


# Ahora construimos nuestra DNN particular, con su propia estructura. En este caso vamos a definir una red
# sencilla de tipo Feed-Forward con tres capas ocultas fully-connected y una ultima capa de salida
class DNN_ENH(_base_net):

    def __init__(self, opt):
        # De nuevo, llamada al INIT de la superclase (padre), en donde establecíamos los atributos orientados
        # al procesado de voz

        #Herencia de clases
        super(DNN_ENH, self).__init__(opt)

        # Computo de dimensiones del input y del target
        self.input_dim = (opt.window_length/2)+1 #ToDo (Numero de puntos del espectro)
        self.target_dim = self.input_dim #ToDo
        self.hidden_layer_size=opt.hidden_units
        
        # Construcción de las capas mediante primitivas de la libreria nn de pytorch 
        # Alternativamente podríamos usar tensores con el cómputo del gradiente activado
        
        #ToDo: Dos capas LSTM, luego una capa oculta y la capa final (mirar lstm y Linear)
        self.LSTM=nn.LSTM(self.input_dim, self.hidden_layer_size, 2,batch_first=True,dropout=opt.dropout_rate)
        self.linear_oculta=nn.Linear(self.hidden_layer_size, self.hidden_layer_size)
        self.fc_end=nn.Linear(self.hidden_layer_size,self.target_dim)
    
        # Capa de dropout (esto si que es mejor que se encarge pytorch)
        #ToDO
        self.drop_layer = nn.Dropout(p=opt.dropout_rate)
            
    # En toda clase DNN de pytorch siempre hay que definir el método "forward" que expresa como se 
    # convierte la entrada en la salida de la red.
    # Este método es llamado automaticamente por el optimizador de la red
    def forward(self, x):
        
        #ToDo: ir operando con el vector x y las capas hasta obtener la máscara de salida (vamos redefiniendo x)
        #Nota: Dropout a la salida de todas las capas menos la ultima. ReLu para la capa oculta y sigmoide para la final
        x, _=self.LSTM(x)
        x=self.drop_layer(x)
        x=self.drop_layer(F.sigmoid(self.linear_oculta(x)))
        x=F.sigmoid(self.fc_end(x))

        return x

In [6]:
####### FUNCIÓN DE PÉRDIDA

# Aunque podría integrarse dentro de la DNN, es conventiente sacar fuera la función de coste, especialmente si
# queremos testear distintos tipos de ellas
class mse_loss(nn.Module):

    def __init__(self):
      # Debe notarse como esta clase hereda de 'nn.Module' como si fuera una especie de red neuronal.
      # De hecho, como tal, le especificamos un método forward
      #Importamos los parametros de la clase nn.Module
      super(mse_loss, self).__init__()
      self.MSE=nn.MSELoss(size_average=None, reduce=None, reduction='none')

      #Si lo aplicamos elemento a elemento y no realizamos media, sigue siendo MSE??

    def forward(self, output, target, pad_mask):
        # Simplemente calcula el MSE entre output y target considerando la máscara de pading (la que usamos para
        # hacer todas las señales del batch del mismo tamaño)
        
        loss_element = self.MSE(output, target)
        loss_sum = torch.sum(loss_element*pad_mask) #ToDo: Enmascarar con el pad_mask y sumar los elementos
        loss = loss_sum/(torch.sum(pad_mask) * len(loss_sum[0,0,:])) #ToDo: Promediar por el numero de elementos utiles (descartar el padding)
        return loss

loss_function = mse_loss()

In [7]:
####### OPTIMIZADOR

# Pytorch integra un buen número de optimizadores que podemos importar
import torch.optim as optim

# El optmizador se aplica sobre una lista de tensores que queremos optimizar, en nuestro caso, esta lista se
# compone de los parametros de nuestra DNN. Por tanto, vamos a instanciar nuestra clase FFN_CTX generando un objeto
# (la red DNN que vamos a entrenar).

# Lo primero que haremos será establecer las opciones de nuestro experimento. Esto generalmente lo haremos mediante
# parametros en la linea de ordenes al llamar al script de python, empleando la libreria 'util.parse_args' para
# procesarlos. Esta librería genera directamente el objeto, que aqui replicaremos a mano de esta forma:
class Opt:
    def __init__(self):
        self.window_length = 512
        self.shift = 256
        self.hidden_units = 1024
        self.dropout_rate = 0.5
        self.early_stop = 20

opt = Opt();

# Con esas opciones instanciamos la red
estimator = DNN_ENH(opt)

# Construyendo finalmente el optimizador (que se aplica sobre los parametros de la red)
optimizer = optim.Adam(estimator.parameters())

In [8]:
####### ENTRENAMIENTO DE LA RED
#####.    1- DataLoaders

# Lo primero que hacemos es preparar un DataLoader. Estos objetos nos permiten paralelizar el trabajo: mientras la GPU
# está ocupada haciendo los pasos forward y backward sobre la DNN con el batch actual, la CPU esta leyendo del disco
# y procesando los datos del siguiente batch.
# Pytorch se encarga del trabajo sucio, tan solo hay que pasar a DataLoader un dataset compatible (esencialmente con
# métodos de inicializacion, devolución de tamaño y de samples como los que hemos definido nuestro 'TimitDataset') y
# un método de construcción de batches compatibles con la red a partir de samples (como el que definimos en nuestra
# DNN '_base_net') 

from torch.utils.data import DataLoader

# Datasets de training y validación
timit_set_train = TimitDataset('../Base_de_datos/Base_datos_1/TIMIT_300/', '../Base_de_datos/Base_datos_2/Noises/', mode='MONO', conj='train')
timit_set_valid = TimitDataset('../Base_de_datos/Base_datos_1/TIMIT_300/', '../Base_de_datos/Base_datos_2/Noises/', mode='MONO', conj='valid')

# DataLoaders de training y validación
# Notese como llamamos al método 'collate_fn' que el objeto 'estimator' de la clase 'FFN_CTX' ha heredado de su clase
# padre '_base_net'
train_dataloader = DataLoader(timit_set_train, batch_size=10, shuffle=True, num_workers=5,
                            collate_fn=estimator.collate_fn)
valid_dataloader = DataLoader(timit_set_valid, batch_size=1, shuffle=False, num_workers=5,
                            collate_fn=estimator.collate_fn)

In [9]:
####### ENTRENAMIENTO DE LA RED
#####.    2- Bucle de optimización

# Antes de nada, vamos a mandar a la GPU (si está disponible) la DNN y la función de coste:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
estimator.to(device)
loss_function.to(device)

import time


# Comenzamos el entrenamiento:

print("[*] Start training...")
num_batch_train = len(train_dataloader)
num_batch_valid = len(valid_dataloader)

num_epochs = 200
save_net = 'model_best.pth.tar'

strike = 0
old_mean_val_distortion = 1e100

# Training epochs
for epoch in xrange(num_epochs):

    # Ciertas capas (como las de dropout, batch_normalization, etc.) se comportan de distinta forma
    # si estamos entrenando la red, o si estamos evaluandola. Para activar el comportamiento adecuado
    # se emplean los métodos 'train' y 'eval' de la clase 'nn.Module'
    
    estimator.train()
    
    avg_loss = 0.0
    start_time = time.time()

    for id, batch in enumerate(train_dataloader):

        # Transferimos los tensores de la CPU a la GPU para realizar el entrenamiento
        features = batch['features'].to(device)
        noisy = batch['noisy'].to(device)
        target = batch['clean'].to(device)
        pad_mask = batch['pad_mask'].to(device)

        # Puesto que el entrenamiento requiere de backpropagation, es necesario que todos los tensores
        # activen sus gradientes. Para no tener que ir uno por uno activamos un contexto de python:
        with torch.set_grad_enabled(True): 

            # Reseteamos los gradientes a 0 (si no se acumularían los nuevos sobre los de la iteracion anterior)
            optimizer.zero_grad()

            # Forward pass
            output_mask = estimator(features)
            output = output_mask * noisy

            loss = loss_function(output, target, pad_mask)
            avg_loss += loss.item() / num_batch_train

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

    # Print loss in the epoch
    print_line = "Epoch: [%2d] time: %4.4f, loss: %.4f" % (epoch + 1, time.time() - start_time, avg_loss)
    print(print_line)

    # Acabada una época, vamos a ver el rendimiento de la red sobre el conjunto de evaluación
    # Ponemos las capas necesarias en modo 'evaluación'
    estimator.eval()
    
    valid_loss = 0.0
    start_time = time.time()

    for id, batch in enumerate(valid_dataloader):

        # Transferimos los tensores de la CPU a la GPU para realizar el entrenamiento
        features = batch['features'].to(device)
        noisy = batch['noisy'].to(device)
        target = batch['clean'].to(device)
        pad_mask = batch['pad_mask'].to(device)

        # Puesto que NO vamos a backpropagar con estos datos, podemos deshabilitar el cómputo de gradientes
        # mejorando el rendimiento
        with torch.no_grad():

            # Forward pass
            output_mask = estimator(features)
            output = output_mask * noisy

            loss = loss_function(output, target, pad_mask)
            valid_loss += loss.item() / num_batch_valid

    # Implementamos un mecanismo de Early-Stopping
    if (valid_loss < old_mean_val_distortion):
        # Si hemos mejorado el entrenamiento, reseteamos el contador de strikes y guardamos el modelo
        print_line = "     Valid: time: %4.4f, valid_loss: %.4f" % (time.time() - start_time, valid_loss)
        print(print_line)
        old_mean_val_distortion = valid_loss
        strike = 0

        # Save the model
        print("[*] Saving model epoch %d..." % (epoch + 1))
        state = {'epoch': epoch + 1, 'state_dict': estimator.state_dict(), 'optimizer': optimizer.state_dict(),
                     'train_loss': avg_loss, 'eval_loss': valid_loss}
        torch.save(state, save_net)

    else:
        # Si NO hemos mejorado el entrenamiento, incrementamos el contador de strikes
        strike += 1

        print_line = "     Valid: time: %4.4f, valid_loss: %.4f *"  % (time.time() - start_time, valid_loss)
        print(print_line)
        # Tras un cierto numero de strikes (también llamado 'patience'), finalizamos el entrenamiento
        if strike > opt.early_stop:
            break


print("[*] Finish training.")

[*] Start training...


/usr/local/lib/python2.7/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


IndexError: ignored

In [ ]:
# Ejercicio final: Implementar un procedimiento de test de la red ya entrenada.
# Evaluar con una métrica objetiva (ya lo vemos mas adelante cuando este todo lo anterior listo)

# Ayuda para cargar red ya entrenada en el estimador (model_load es el fichero donde tenemos los parametros)
#checkpoint = torch.load(model_load)
#estimator.load_state_dict(checkpoint['state_dict'])